In [6]:
import json
import pandas as pd

In [2]:
data_path = 'data/doc2dial/'
file_path = data_path + 'doc2dial_dial_train.json'
with open(file_path, 'r') as f:
    data = json.load(f)

In [3]:
domains = ['dmv', 'ssa', 'va', 'studentaid']

In [25]:
def run():
    df = pd.DataFrame(columns=['question', 'answer', 'domain', 'doc_id', 'references', 'dial_id'])
    cnt = 0
    for domain in domains:
        sub_dict = data['dial_data'][domain]
        #iterate sub_dict
        for key in sub_dict.keys():
            #iterate turns
            dial_list = sub_dict[key]
            for dial in dial_list:
                dial_id = dial['dial_id']
                for turn in dial['turns']:
                    role = turn['role']
                    ref = turn['references']

                    if role == 'user':
                        question =  turn['utterance']
                    elif role == 'agent':
                        answer =  turn['utterance']
                        df.loc[len(df)] = [question, answer, domain, key, ref, dial_id]
                        # qa_list.append([question, answer, domain, ref])
                        cnt += 1
                    else:
                        print('Error: No such role')
                        
    print('Total QA pairs: ', cnt)
    df.to_csv(data_path + 'doc2dial_qa.csv', index=False)           
            


In [26]:
run()

Total QA pairs:  21998


In [28]:
df = pd.read_csv(data_path + 'doc2dial_qa.csv')
df.head(10)

,question,answer,domain,doc_id,references,dial_id
0,"Hello, I forgot o update my address, can you h...","hi, you have to report any change of address t...",dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '6', 'label': 'solution'}, {'sp_id'...",9f44c1539efe6f7e79b02eb1b413aa43
1,Can I do my DMV transactions online?,"Yes, you can sign up for MyDMV for all the onl...",dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '56', 'label': 'solution'}]",9f44c1539efe6f7e79b02eb1b413aa43
2,"Thanks, and in case I forget to bring all of t...",This happens often with our customers so that'...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '49', 'label': 'solution'}, {'sp_id...",9f44c1539efe6f7e79b02eb1b413aa43
3,"Ok, and can you tell me again where should I r...",Sure. Any change of address must be reported t...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '6', 'label': 'solution'}, {'sp_id'...",9f44c1539efe6f7e79b02eb1b413aa43
4,Can you tell me more about Traffic points and ...,Traffic points is the system used by DMV to tr...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '41', 'label': 'solution'}, {'sp_id...",9f44c1539efe6f7e79b02eb1b413aa43
5,Will you keep me informed about when my insura...,Yes. The New York State requires every motoris...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '24', 'label': 'solution'}, {'sp_id...",88ecf840ea87f8c53faff15d4f0bb214
6,Is it needed that my address is always up to d...,"Yes, It is required. If not we won't be able t...",dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '35', 'label': 'solution'}, {'sp_id...",88ecf840ea87f8c53faff15d4f0bb214
7,Can you tell me more about the traffic points ...,Yes. This is the system DM uses to keep a reco...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '41', 'label': 'solution'}, {'sp_id...",88ecf840ea87f8c53faff15d4f0bb214
8,It's always a hassle to bring all the required...,This happens quite often with our customers wh...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '49', 'label': 'solution'}, {'sp_id...",88ecf840ea87f8c53faff15d4f0bb214
9,In case I have a problem with my insurance and...,"In that case, we must suspend the vehicle regi...",dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0,"[{'sp_id': '32', 'label': 'solution'}, {'sp_id...",88ecf840ea87f8c53faff15d4f0bb214


In [34]:
#mapping doc_id to dail_id
df[['doc_id', 'dial_id']]
#remove duplicates
df = df.drop_duplicates(subset=['doc_id', 'dial_id'])

3474

In [35]:
len(df[['doc_id', 'dial_id']])

3474

In [42]:
df[['dial_id', 'doc_id']].head(10)

,dial_id,doc_id
0,9f44c1539efe6f7e79b02eb1b413aa43,Top 5 DMV Mistakes and How to Avoid Them#3_0
5,88ecf840ea87f8c53faff15d4f0bb214,Top 5 DMV Mistakes and How to Avoid Them#3_0
10,3c079944cf36c05b45f668669cc3301b,Top 5 DMV Mistakes and How to Avoid Them#3_0
15,9fa8c68e2f211edad9fb65eb5897d600,Top 5 DMV Mistakes and How to Avoid Them#3_0
20,423bd6a66d14c22f8be69add96679255,Top 5 DMV Mistakes and How to Avoid Them#3_0
25,15aa88cc47d6c698af75ce5a00cc7c04,Top 5 DMV Mistakes and How to Avoid Them#3_0
31,eec4a276b8f0aebb4069d0dbca94b30f,Top 5 DMV Mistakes and How to Avoid Them#3_0
37,8b326c47087dfbec10f9ad1e4df1df2b,Top 5 DMV Mistakes and How to Avoid Them#3_0
42,494fe90938a0d159620d0229fbe059b2,Top 5 DMV Mistakes and How to Avoid Them#3_0
49,1f84f158224bcc4b92bf466b7f0986b7,Top 5 DMV Mistakes and How to Avoid Them#3_0


In [46]:
df[['dial_id', 'doc_id']]
#build a dictionary mapping dial_id to doc_id
dial2doc = {}
for index, row in df.iterrows():
    dial2doc[row['dial_id']] = row['doc_id']
    
#build a dictionary mapping doc_id to dial_id (one to many)
doc2dial = {}
for index, row in df.iterrows():
    if row['doc_id'] in doc2dial:
        doc2dial[row['doc_id']].append(row['dial_id'])
    else:
        doc2dial[row['doc_id']] = [row['dial_id']]


In [52]:
#write dial2doc and doc2dial to json file
output = {'dial2doc':dial2doc, 'doc2dial':doc2dial}
with open(data_path + 'doc2dial_mapping_train.json', 'w') as f:
    json.dump(output, f)